# A simple REST service w/CRUD

We will go over a small example REST service to highlight a couple of python features

- Intro to pyproject.toml
- Demo of getting existing data, creating new data, and showing existing data
    - Show the code
    - Start the service in reload mode
    - Go to the swagger page
    - Run example
    - Run example with the httpx client

## The pyproject.toml file

Python has started strongly leaning into the TOML file world over yaml. A TOML file will unambiguously map to...well, a
map. The pyproject.toml has become standardized in [PEP-518](https://peps.python.org/pep-0518/) and is used by many
python tooling like poetry, autopep8, pyflake, hatch, pipenv, and many others.

We are using the poetry tool, and it has some important metadata extras.  

- It does the work that would normally be done in a requirements.txt file
    - the dependency resolver is more sophisticated than the default pip
- replaces `setuptools.py` to build a wheel and upload to PyPI

The pyproject.toml file's most import parts are:

- The version of the project
- The dependencies
    - Dependencies can declare a range, or exact specifications
    - Dependencies can be optional and installed with a `-E` extras flag
    - Dependencies can also be installed in groups like for development with `-G dev`
- Can specify alternative repositories (eg private repos)

## Demo of ares: The code

Show off a little demo of the beginnings of a roleplaying game engine.  It can create and retrieve characters from a
local duckdb database.  duckdb is like sqlite on steroids.  Although it's meant for OLAP rather than OLTP apps, it has
full ACID guarantees, so it can be used for OLTP also.

- Show the initialization of the database
- Show the functions in service.py
- Show the model in db/characters/characters.py

In [ ]:
from pathlib import Path
import duckdb as dd
from duckdb import DuckDBPyConnection
from fastapi import FastAPI
import uvicorn

from ares.models.character.character import Character, init_char_pq_path
import pyarrow as pa

# Initializing the database

We use duckdb to store and query the data.  duckdb is extremely fast (faster than Spark by a wide margin on data that 
can fit on a single node).  Perhaps most interestingly, it can query JSON data that's been stored in a newline delimited
format (eg NDJSON aka JSONL).

Here, if we don't already have a database, it will be created.  Note the `init_df` variable which _looks_ like it is not
being used.  But actually it is.  Duckdb will use the dataframe from the seeded initial parquet file (with the embedded
schema) and be able to use it like SQL Table.

> duckdb persistence
>
> By default, you do not have to give a name to connect, and it will use an in-memory database.  All data will be lost
> when the program stops though

If the db file does exist, then we load the `char_db` table.  A `testing` inner function will create a new random
character each time we start up the service.  This is an example of a python inner function, which is handy when you
want a _private_ function that is not accessible to outside code (technically, you can still get at it, but don't be a
bad programmer).

In [ ]:
# Create or open the database our service will use
def init_db(db_name: str = "ares.db") -> DuckDBPyConnection:
    if not Path(db_name).exists():
        conn = dd.connect(db_name)
        init_df = conn.read_parquet(f"{init_char_pq_path}")
        conn.execute("CREATE TABLE char_db as SELECT * FROM init_df")
    else:
        conn = dd.connect(db_name)
        print(conn.sql("SHOW ALL TABLES"))
        print(conn.sql("SELECT * FROM char_db"))


    def testing():
        example = Character.random_character()
        schema = Character.arrow_schema()
        tbl = pa.Table.from_pylist([example.model_dump()], schema=schema)
        sql_cmd = f"INSERT INTO char_db SELECT * FROM tbl RETURNING *"
        print(sql_cmd)
        df = conn.sql(sql_cmd)
        print(df)

    testing()
    return conn

## The actual REST methods

This is how the actual methods are implemented.

In [ ]:
# Create the FastAPI application
app = FastAPI()

# /v1/characters
char_ept = "/v1/characters/"

# Create the database connection
conn = init_db()


@app.post(char_ept)
async def post_character(char: Character):
    model = char.model_dump()
    print(model)
    schema = Character.arrow_schema()
    tbl = pa.Table.from_pylist([model], schema=schema)
    sql_cmd = f"INSERT INTO char_db SELECT * FROM tbl RETURNING *"
    print(sql_cmd)
    df = conn.sql(sql_cmd)
    print(df)


@app.get(f"{char_ept}/{{uid}}")
async def get_character(uid: str):
    sql_cmd = f"SELECT * FROM char_db WHERE uid = '{uid}'"
    df = conn.sql(sql_cmd)
    batch = df.to_arrow_table()
    data = batch.to_pylist()[0]
    return data

